In [24]:
import pandas as pd
import numpy as np
import re

file_path = 'splitted.csv'
df = pd.read_csv(file_path).drop('Unnamed: 0', axis=1).dropna(how='all', axis=0).dropna(how='all', axis=1)
file_path = '_SELECT_r_p_v_vd_FROM_record_r_JOIN_person_p_ON_r_person_index_p_202408261231.csv'
df_joined = pd.read_csv(file_path).drop('Unnamed: 0', axis=1)
for col in df.select_dtypes(include=['object']):
    df[col] = df[col].str.strip()
df.replace('[nan]', np.nan, inplace=True)

<ipython-input-24-69a4946e59c6>:8: DtypeWarning: Columns (4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_joined = pd.read_csv(file_path).drop('Unnamed: 0', axis=1)


In [25]:
def replace_nan_array(val):
    if isinstance(val, np.ndarray):
        if val.size == 1 and np.isnan(val[0]):
            return np.nan
    return val

def match_or_nan(a, b):
    # Function to check if two values are equal or both are NaN
    return (a == b) | (pd.isna(a) & pd.isna(b))

def check_issues(row):
    issues = []
    if not pd.isna(row['Emb_date']) and not pd.Series(row['Emb_date']).str.match(r'^\d{2}/\d{2}/\d{4}$', na=False).bool():
        issues.append('emb_date')
    if pd.isna(row['Emb_date']):
        issues.append('emb_date')
    if not pd.isna(row['Disemb_date']) and not pd.Series(row['Disemb_date']).str.match(r'^\d{2}/\d{2}/\d{4}$', na=False).bool():
        issues.append('disemb_date')
    if pd.isna(row['Disemb_date']):
        issues.append('Disemb_date')
    if pd.isna(row['Emb_loc']):
        issues.append('emb_loc')
    if pd.isna(row['Disemb_loc']):
        issues.append('disemb_loc')
    return issues


def convert_to_ddmmyyyy(date_value):
    if pd.isna(date_value) or str(date_value).lower() in ['nan', '[nan]']:
        return np.nan
    try:
        # Parse the date and reformat to dd/mm/yyyy
        return pd.to_datetime(date_value).strftime('%d/%m/%Y')
    except (ValueError, TypeError):
        return date_value


def extract_details(text):
    # Patterns for embarkation
    embark_patterns = {
        'embarqué': {                                                                                           
            'date': r'\b(?:embarqué|rembarqué|embarquée|rembarquée|embarqués|rembarqués|embarquées|rembarquées) (?:(?!\b(?:débarqué|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées|resté)\b).)*le (\d{2}/\d{2}/\d{4})(?=\s|\n|$| ---|,)',
            'location': r'\b(?:embarqué|rembarqué|embarquée|rembarquée|embarqués|rembarqués|embarquées|rembarquées) à ([\w\s\'-îÎ]+?)(?=\s+\b(?:mort[es]?|débarqué[es]?|déserté[es]?|passé[es]?|resté[es]?|tombé[es]?)\b| le \d{2}/\d{2}/\d{4}|,| ---|\n|$)'
        },
        'a fait la': {
            'date': r'remplacement du (\d{2}/\d{2}/\d{4})',
            'location': r'\b(?:a|à) fait la campagne de ([\w\s\'-îÎ]+?) à ([\w\s\'-îÎ]+)'
        },
        'supplément': {
            'date': r'\bsupplément à (?!\b(?:débarqué|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées)\b)[\w\s\'-îÎ]+? du (\d{2}/\d{2}/\d{4})',
            'location': r'\bsupplément à ([\w\s\'-îÎ]+?) du \d{2}/\d{2}/\d{4}'
        },
        'remplacement': {
            'date': r'\bremplacement (?:au|à)?(?!\b(?:débarqué|débarquée|débarqués|débarquées|déserté|désertée|désertés|désertées|mort|morte|morts|mortes|passé|passée|passés|passées)\b)[\w\s\'-îÎ]+?(?: le| du) (\d{2}/\d{2}/\d{4})',
            'location': r'\bremplacement (?:au|à)? ([\w\s\'-îÎ]+?)(?: en| le| du|\n|,| ---|$)'
        },
        'trouvé': {
            'date': r'\btrouvé[es]? caché[es]? à bord(?:(?!\b(?:débarqué[es]?|déserté[es]?|mort[es]?|passé[es]?)\b)[^\n])*? le (\d{2}/\d{2}/\d{4})',
            'location': r'\btrouvé[es]? caché[es]? à bord (?:après le départ de|le \d{2}/\d{2}/\d{4})\s+((?!\b(?:débarqué[es]?|sert|déserté[es]?|mort[es]?|passé[es]?)\b)[\w\s\'-îÎ]+?)(?=\s+\b(?:le\s+\d{2}/\d{2}/\d{4}|---|,|\n|$|\b(?:débarqué[es]?|sert|déserté[es]?|mort[es]?|passé[es]?)\b))'
        }
    }

    # Patterns for disembarkation
    disembark_patterns = {
        'débarqué': {
            'date': r'\bdébarqué[es]?.*?(?:à|au|furtivement à|malade après être tombé du haut mal à)? (?:désarmement à |malade à |malade et mort à l\'hôpital (?:de|du) |malade à l\'hôpital (?:de|du) )?[\w\s\'-îÎ]+? le (\d{2}/\d{2}/\d{4})',
            'location': r'\bdébarqué[es]? (?:au |à |furtivement à |malade après être tombé du haut mal à )?(?:désarmement à |malade à l\'hôpital (?:de|du) |malade à |malade et mort à l\'hôpital (?:de|du) )?([\w\s\'-îÎ]+?)(?= le \d{2}/\d{2}/\d{4}| ---|,|\n|$| mort[es]?)'
        },
        'déserté': {
            'date': r'\bdéserté[es]? (?:à [\w\s\'-îÎ]+|sur le vaisseau de côte le [\w\s\'-îÎ]+|en [\w\s\'-îÎ]+) le (\d{2}/\d{2}/\d{4})',
            'location': r'\bdéserté[es]? (?:à|en|au départ de) ([\w\s\'-îÎ]+?)(?: le|\n|,| ---|$)'
        },
        'mort en mer': {
            'date': r'\b(?:mort[es]? (?:en mer|noyé en [\w\s\'-îÎ]+|du [\w\s\'-îÎ]+ en [\w\s\'-îÎ]+|à l\'hôpital de [\w\s\'-îÎ]+|à la ration|à [\w\s\'-îÎ]+)|tombé à la mer et mort noyé) le (\d{2}/\d{2}/\d{4})',
            'location': r'\bmort[es]? (?:en mer|à l\'hôpital (?:du|de)|à la ration|à) ([\w\s\'-îÎ]+?)(?= le \d{2}/\d{2}/\d{4})'
        },
        'a fait la': {
            'date': r'levé[es]? du (\d{2}/\d{2}/\d{4})',
            'location': r'\b(?:a|à) fait la campagne de [\w\s\'-îÎ]+? à ([\w\s\'-îÎ]+?)(?=\s*(?:du|le) \d{2}/\d{2}/\d{4}|\s*---|,|en|\n|$)'
        },
        'passé': {
            'date': r'\bpassé[es]? sur la [\w\s\'-îÎ]+ en rade de [\w\s\'-îÎ]+ le (\d{2}/\d{2}/\d{4})',
            'location': r'\bpassé[es]? sur la [\w\s\'-îÎ]+ en rade de ([\w\s\'-îÎ]+)(?=\s+(?:le\s+\d{2}/\d{2}/\d{4}|,| ---|\n|$))'
        },
        'resté': {
            'date': r'\bresté[es]? (?:malade au départ de|malade à l\'hôpital de|à terre au départ de|à terre malade au départ de|à terre malade à|à|en|au départ de) [\w\s\'-îÎ]+? le\s*(\d{2}/\d{2}/\d{4})(?: rejoint|\n|,| ---|$)',
             'location': r'\bresté[es]? (?:malade au départ de|malade à l\'hôpital de|à terre au départ de|à terre malade au départ de|à terre malade à|à|en|au départ de) ([\w\s\'-îÎ]+?)(?: le| rejoint|\n|,| ---|$)'

        }
    }

    # Initialize results
    embark_location = pd.NA
    embark_date = pd.NA
    disembark_location = pd.NA
    disembark_date = pd.NA

    # Initialize match variables
    embark_date_match = None
    embark_location_match = None
    disembark_date_match = None
    disembark_location_match = None

    # Determine which embark patterns to use
    if 'embarqué' in text.lower() or 'rembarqué' in text.lower():
        embark_date_match = re.search(embark_patterns['embarqué']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['embarqué']['location'], text, flags=re.IGNORECASE)
    elif 'fait la campagne' in text.lower():
        embark_date_match = re.search(embark_patterns['a fait la']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['a fait la']['location'], text, flags=re.IGNORECASE)
    elif 'supplément' in text.lower():
        embark_date_match = re.search(embark_patterns['supplément']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['supplément']['location'], text, flags=re.IGNORECASE)
    elif 'remplacement' in text.lower():
        embark_date_match = re.search(embark_patterns['remplacement']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['remplacement']['location'], text, flags=re.IGNORECASE)
    elif 'trouvé' in text.lower():
        embark_date_match = re.search(embark_patterns['trouvé']['date'], text, flags=re.IGNORECASE)
        embark_location_match = re.search(embark_patterns['trouvé']['location'], text, flags=re.IGNORECASE)        

    # Extract embark information
    if embark_date_match:
        embark_date = embark_date_match.group(1)
    if embark_location_match:
        embark_location = embark_location_match.group(1)

    # Determine which disembark patterns to use
    if 'débarqué' in text.lower():
        disembark_date_match = re.search(disembark_patterns['débarqué']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['débarqué']['location'], text, flags=re.IGNORECASE)
    elif 'déserté' in text.lower():
        disembark_date_match = re.search(disembark_patterns['déserté']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['déserté']['location'], text, flags=re.IGNORECASE)
    elif 'mort' in text.lower():
        disembark_date_match = re.search(disembark_patterns['mort en mer']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['mort en mer']['location'], text, flags=re.IGNORECASE)
    elif 'passé' in text.lower():
        disembark_date_match = re.search(disembark_patterns['passé']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['passé']['location'], text, flags=re.IGNORECASE)
    elif 'fait la campagne' in text.lower():
        disembark_date_match = re.search(disembark_patterns['a fait la']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['a fait la']['location'], text, flags=re.IGNORECASE)
    elif 'resté' in text.lower():
        disembark_date_match = re.search(disembark_patterns['resté']['date'], text, flags=re.IGNORECASE)
        disembark_location_match = re.search(disembark_patterns['resté']['location'], text, flags=re.IGNORECASE)

    # Extract disembark information
    if disembark_date_match:
        disembark_date = disembark_date_match.group(1)
    if disembark_location_match:
        disembark_location = disembark_location_match.group(1)
        if len(disembark_location_match.groups()) > 1 and disembark_location_match.group(2):
            disembark_location += f", {disembark_location_match.group(2)}"
    if 'mort à bord' in text.lower() or 'morte à bord' in text.lower() or 'morts à bord' in text.lower() or 'mortes à bord' in text.lower():
        disembark_location = 'on board'
    elif 'mort en mer' in text.lower() or 'morte en mer' in text.lower() or 'morts en mer' in text.lower() or 'mortes en mer' in text.lower():
        disembark_location = 'at sea'
    if 'née en mer' in text.lower() or 'né en mer' in text.lower() or 'nées en mer' in text.lower() or 'nés en mer' in text.lower():
        embark_location = 'at sea'
    elif 'née à bord' in text.lower() or 'né à bord' in text.lower() or 'nées à bord' in text.lower() or 'nés à bord' in text.lower():
        embark_location = 'on board'
    
            

    return embark_location, embark_date, disembark_location, disembark_date

def fill_emb_loc_for_rembarque(df):
    # Sort the dataframe by Last Name, Emb_date, and Disemb_date
    df_sorted = df.sort_values(by=['Last Name', 'Emb_date', 'Disemb_date']).reset_index(drop=True)
    
    # Iterate through the sorted dataframe
    for i in range(1, len(df_sorted)):
        current_row = df_sorted.iloc[i]
        previous_row = df_sorted.iloc[i - 1]
        
        # Check if Remarks is a string and contains the keyword "rembarqué"
        if pd.isna(current_row['Emb_loc']) and isinstance(current_row['Remarks'], str) and 'rembarqué' in current_row['Remarks'].lower():
            # Set the current row's Emb_loc to the previous row's Disemb_loc
            df_sorted.at[i, 'Emb_loc'] = previous_row['Disemb_loc']
    
    return df_sorted

In [26]:
# Extracting info from Remarks

df['Emb_loc'] = None
for index, row in df.iterrows():
    remark = re.sub(r'\s+', ' ', str(row['Remarks']).replace('\n', ' '))
    
    emb_date = row['Emb_date']
    disemb_date = row['Disemb_date']
    disemb_loc = row['Disemb_loc']
    embark_location, embark_date, disembark_location, disembark_date = extract_details(remark)
    
    df.at[index, 'Emb_loc'] = embark_location
    if pd.notna(disembark_date):
        df.at[index, 'Disemb_date'] = disembark_date
    if pd.notna(embark_date):
        df.at[index, 'Emb_date'] = embark_date
    if pd.notna(disembark_location):
        df.at[index, 'Disemb_loc'] = disembark_location

# Apparently, fill emb_loc for re-embark
df = fill_emb_loc_for_rembarque(df)

In [27]:
# replacing "armement"

for index, row in df.iterrows():
    if pd.notna(row['Remarks']):
        if isinstance(row['Disemb_loc'], str) and 'armement' in row['Disemb_loc'].lower():
            loc = list(df_joined[
                ((df_joined['last_name'] == row['Last Name']) | (df_joined['last_name'].isna() & pd.isna(row['Last Name']))) &
                ((df_joined['first_name'] == row['First Name']) | (df_joined['first_name'].isna() & pd.isna(row['First Name']))) &
                ((df_joined['emb_date'] == row['Emb_date']) | (df_joined['emb_date'].isna() & pd.isna(row['Emb_date'])))
            ]['steps'])
            if len(loc) > 0:
                df.at[index, 'Disemb_loc'] = loc[0].split("-")[-1]

        if isinstance(row['Emb_loc'], str) and 'armement' in row['Emb_loc'].lower():
            loc = list(df_joined[
                ((df_joined['last_name'] == row['Last Name']) | (df_joined['last_name'].isna() & pd.isna(row['Last Name']))) &
                ((df_joined['first_name'] == row['First Name']) | (df_joined['first_name'].isna() & pd.isna(row['First Name']))) &
                ((df_joined['emb_date'] == row['Emb_date']) | (df_joined['emb_date'].isna() & pd.isna(row['Emb_date'])))
            ]['steps'])
            if len(loc) > 0:
                df.at[index, 'Emb_loc'] = loc[0].split(" ")[0]

In [28]:
df['Emb_date'] = df['Emb_date'].apply(convert_to_ddmmyyyy)
df['Disemb_date'] = df['Disemb_date'].apply(convert_to_ddmmyyyy)
df['Emb_date'] = df['Emb_date'].apply(lambda x: x if isinstance(x, str) else np.nan)
df['Disemb_date'] = df['Disemb_date'].apply(lambda x: x if isinstance(x, str) else np.nan)

<ipython-input-25-ce7700b488f2>:33: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')


In [29]:
# Add emb_class and disemb_class

df['emb_class'], df['disemb_class'] = None, None
for index, row in df.iterrows():
    if not isinstance(row['Remarks'], str):
        continue
    if 'clandestin' in row['Remarks'] or 'caché' in row['Remarks']:
        df.at[index, 'emb_class'] = 304
    elif 'rembarqué' in row['Remarks']:
        df.at[index, 'emb_class'] = 306
    elif 'fait la campagne' in row['Remarks'] or 'armement' in row['Remarks'] or 'embarqué' in row['Remarks']:
        df.at[index, 'emb_class'] = 301
    elif 'remplacement' in row['Remarks'] or 'supplément' in row['Remarks']:
        df.at[index, 'emb_class'] = 302
    elif 'resté à terre' in row['Remarks']:
        df.at[index, 'emb_class'] = 308
    elif 'renversement' in row['Remarks'] or 'vient' in row['Remarks']:
        df.at[index, 'emb_class'] = 303
    elif 'né' in row['Remarks'] or 'née' in row['Remarks']:
        df.at[index, 'emb_class'] = 305
    else:
        df.at[index, 'disemb_class'] = 309
        
    if 'mort' in row['Remarks']:
        df.at[index, 'disemb_class'] = 305
    elif 'fait la campagne' in row['Remarks']:
        df.at[index, 'disemb_class'] = 301
    elif 'resté malade' in row['Remarks'] or 'resté à l\'hôpital' in row['Remarks']:
        df.at[index, 'disemb_class'] = 306
    elif 'débarqué' in row['Remarks']:
        if pd.notna(row['Emb_loc']) and pd.notna(row['Disemb_loc']) and row['Emb_loc'] == row['Disemb_loc']:
            df.at[index, 'disemb_class'] = 301
        else:
            df.at[index, 'disemb_class'] = 302
    elif 'passé sur' in row['Remarks']:
        df.at[index, 'disemb_class'] = 303
    elif 'échapé' in row['Remarks'] or 'fugitif' in row['Remarks'] or 'déserté' in row['Remarks']:
        df.at[index, 'disemb_class'] = 304
    elif 'prise par le' in row['Remarks']:
        df.at[index, 'disemb_class'] = 307
    elif 'resté à terre' in row['Remarks'] or 'restée à terre' in row['Remarks']:
        df.at[index, 'disemb_class'] = 308
    else:
        df.at[index, 'disemb_class'] = 309

In [30]:
# Well... filling based on re-embark???

reembark_rows = df[(df['Remarks'].str.contains('rembarqué', na=False, case=False)) & (df['Emb_loc'].isna())]

# Step 5: Process each row with the "rembarqué" keyword individually
for index, row in reembark_rows.iterrows():
    # Find rows with matching 'Last Name', 'First Name', and 'Function', handling NaN values
    matching_rows = df[
        match_or_nan(df['Last Name'], row['Last Name']) &
        match_or_nan(df['First Name'], row['First Name']) &
        match_or_nan(df['Function'], row['Function'])
    ]

    # Sort the matching rows by Emb_date
    sorted_group = matching_rows.sort_values(by=['Emb_date', 'Disemb_date'])

    # Iterate through the sorted group to assign emb_loc based on previous Disemb_loc
    previous_disemb_loc = None
    for i, (sorted_index, sorted_row) in enumerate(sorted_group.iterrows()):
        if sorted_index == index:
            if previous_disemb_loc:
                df.at[sorted_index, 'Emb_loc'] = previous_disemb_loc

        # Update previous_disemb_loc for the next row in the sequence
        previous_disemb_loc = sorted_row['Disemb_loc']
        
for _, group in df.groupby(['Last Name', 'First Name', 'Function'], dropna=False):
    sorted_group = group.sort_values(by=['Emb_date', 'Disemb_date'])
    for i, (index, row) in enumerate(sorted_group.iterrows()):
        if pd.isna(row['Disemb_loc']) and i < len(sorted_group) - 1:
            next_row = sorted_group.iloc[i + 1]
            df.at[index, 'Disemb_loc'] = next_row['Emb_loc']

In [31]:
# Fill remaining from database

for index, row in df.iterrows():
    if pd.notna(row['Remarks']):
        if pd.isna(row['Emb_date']):
            df.at[index, 'Emb_date'] = list(df_joined[
                ((df_joined['last_name'] == row['Last Name']) | (df_joined['last_name'].isna() & pd.isna(row['Last Name']))) &
                ((df_joined['remarks'] == row['Remarks']) | (df_joined['remarks'].isna() & pd.isna(row['Remarks']))) &
                ((df_joined['first_name'] == row['First Name']) | (df_joined['first_name'].isna() & pd.isna(row['First Name'])))
            ]['date_voy_begin'])[0]
            
        if pd.isna(row['Emb_loc']):
            df.at[index, 'Emb_loc'] = list(df_joined[
                ((df_joined['last_name'] == row['Last Name']) | (df_joined['last_name'].isna() & pd.isna(row['Last Name']))) &
                ((df_joined['remarks'] == row['Remarks']) | (df_joined['remarks'].isna() & pd.isna(row['Remarks']))) &
                ((df_joined['first_name'] == row['First Name']) | (df_joined['first_name'].isna() & pd.isna(row['First Name'])))
            ]['start_loc'])[0]
            
        if pd.isna(row['Disemb_date']):
            df.at[index, 'Disemb_date'] = list(df_joined[
                ((df_joined['last_name'] == row['Last Name']) | (df_joined['last_name'].isna() & pd.isna(row['Last Name']))) &
                ((df_joined['remarks'] == row['Remarks']) | (df_joined['remarks'].isna() & pd.isna(row['Remarks']))) &
                ((df_joined['first_name'] == row['First Name']) | (df_joined['first_name'].isna() & pd.isna(row['First Name'])))
            ]['date_voy_end'])[0]
            
        if pd.isna(row['Disemb_loc']) or row['Disemb_loc']=='ration':
            df.at[index, 'Disemb_loc'] = list(df_joined[
                ((df_joined['last_name'] == row['Last Name']) | (df_joined['last_name'].isna() & pd.isna(row['Last Name']))) &
                ((df_joined['remarks'] == row['Remarks']) | (df_joined['remarks'].isna() & pd.isna(row['Remarks']))) &
                ((df_joined['first_name'] == row['First Name']) | (df_joined['first_name'].isna() & pd.isna(row['First Name'])))
            ]['last_loc'])[0]

In [32]:
df['Emb_date'] = df['Emb_date'].apply(convert_to_ddmmyyyy)
df['Disemb_date'] = df['Disemb_date'].apply(convert_to_ddmmyyyy)
df['Emb_date'] = df['Emb_date'].apply(lambda x: x if isinstance(x, str) else np.nan)
df['Disemb_date'] = df['Disemb_date'].apply(lambda x: x if isinstance(x, str) else np.nan)
df['Emb_loc'] = df['Emb_loc'].apply(lambda x: x if isinstance(x, str) else np.nan)
df['Disemb_loc'] = df['Disemb_loc'].apply(lambda x: x if isinstance(x, str) else np.nan)
df.replace('[nan]', np.nan, inplace=True)

<ipython-input-25-ce7700b488f2>:33: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  return pd.to_datetime(date_value).strftime('%d/%m/%Y')


In [33]:
df['issued'] = df.apply(check_issues, axis=1)
for index, row in df.iterrows():
    if row['Disemb_loc'] == 'ration' or row['Disemb_loc'] == 'mer et mort noyé':
        df.at[index, 'issued'].append('disemb_loc')

In [34]:
df.to_csv('combined.csv')